# Results

## 1) Actor coefficients with and without confounders.

Load model and coefficients from memory, compare for set of actors. How does it hold up against Blei's claims? Most corrected actors, most over valued, most undervalued actors?

In [32]:
import pandas as pd
import numpy as np
import pickle
import pyro
# Data Loader
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../box_office/')
from data_loader import load_tensor_data

In [2]:
# Load data, just actors for now
_, _ , actors = load_tensor_data("../data/ohe_movies.csv")

In [3]:
# Load params
with open('params.pickle', 'rb') as f:
    deconf_params = pickle.load(f) 
    
# Get params from standard regression
pyro.get_param_store().load("linreg_params")
linreg_params = {}
for name, value in pyro.get_param_store().items():
    linreg_params[name] = pyro.param(name)

In [19]:
linreg_mean = linreg_params['auto_loc'] # Remove bias and intercept from here
deconf_mean = deconf_params['qweight_mean'] # Remove the 30 latent dims (bias intercept are already different params)

In [29]:
diff_vector = linreg_mean[:-2] - deconf_mean[0][:-30]   # w/o conf coeff - with conf coeff
# If this is positive, then the actor was over valued. confounding decreased their causal effect on revenue

In [38]:
pd.DataFrame(list(zip(actors, diff_vector.detach()), columns = ["Actors", "Causal Effect Delta after Deconfounding"]))

TypeError: list() takes no keyword arguments

## 2) Does this model better prediction accuracy in any way?

Predict revenue with same movies data from both the models and check RMSE scores.

## 3) Counterfactual queries

Design some queries for e.g What if Batman was made with Nicolas Cage instead of Ben Affleck. The most underrated actor vs the most expensive in one of his movies, how much would revenue change. Does it make sense?